## 3 - Neural Machine Translation by Jointly Learning to Align and Translate

> Ở notebook thứ 3 này, chúng ta sẽ cùng implementing model từ paper [Neural Machine Translation by Jointly Learning to Align and Translate](https://arxiv.org/pdf/1409.0473.pdf). Model này đạt được perplexity xấp xỉ 27, so với 34 từ các model trước.

### Introduction

> Đây là mô hình encoder-decoder được sử dụng từ các notebooks:

> ![figure1](./images/3.seq2seq.png)

> Ở model trước, chúng ta cùng thiết lập một kiến trúc để giảm việc nén thông tin bằng cách truyền context vector z vào decoder ở mỗi thời điểm. Và, ta truyền cả context vector và embedded input word, $\bold{d(y_t)}$ cùng với hidden state $s_t$ vào linear layer, f để đưa ra dự đoán.

![figure2](./images/3.seq2seq_2.png)

> Mặc dù giảm được việc nén thông tin, context vector vẫn phải cần lưu trữ thông tin về source sentence.

> Trong notebook này, chúng ta sẽ xây dựng một model xóa bỏ việc đè nén thông tin bằng cách cho phép decoder quan sát toàn bộ source sentence (thông qua các hidden states của nó) ở mỗi bước decoding. Và, chúng ta sẽ sử dụng *attention*. Chi tiết về *attention* sẽ được trình bày ở mục sau.

### Preparing Data

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchtext.legacy.datasets import Multi30k
from torchtext.legacy.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time

ModuleNotFoundError: No module named 'torchtext.legacy'

> Thiết lập random seeds.

In [ ]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

> Load model tokenize tiếng Anh và tiếng Đức của spaCy.

In [ ]:
spacy_de = spacy.load('de_core_news_sm')
spacy_en = spacy.load('en_core_web_sm')

> Tạo tokenizers.

In [ ]:
def tokenize_de(text):
    """
    Tokenizes German text from a string into a list of strings
    """
    return [tok.text for tok in spacy_de.tokenizer(text)]

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [2]:
SRC = Field(tokenize = tokenize_de, 
            init_token = '', 
            eos_token = '', 
            lower = True)

TRG = Field(tokenize = tokenize_en, 
            init_token = '', 
            eos_token = '', 
            lower = True)

NameError: name 'Field' is not defined

> Load data.

In [ ]:
train_data, valid_data, test_data = Multi30k.splits(exts = ('.de', '.en'), 
                                                    fields = (SRC, TRG))

> Build vocabulary.

In [ ]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)

### Building the Seq2Seq Model

##### Encoder

> Đầu tiên, ta sẽ xây dựng encoder. Tương tự như model ở notebook trước, chúng ta chỉ sư dụng một layer GRU, tuy nhiên bây giờ ta sẽ sử dụng *bidirectional RNN*. Với *bidirectional RNN*, ta có 2 lớp RNNs ở mỗi layer. Quá trình foward của layer RNN đi từ trái sang phải (green), quá trình backward đi từ phải sang trái (teal). Trong pytorch, ta chỉ cần set `bidirectional = True` để sử dụng *bidirectional RNN*.

![figure3](./images/3.seq2seq_bidirectional.png)

> Giờ đây, chúng ta có: <br> <br>
> $\begin{aligned} h_t^{\rightarrow} &=\text { EncoderGRU } \rightarrow\left(e\left(x_t^{\rightarrow}\right), h_{t-1}^{\rightarrow}\right) \\ h_t^{\leftarrow} &=\text { EncoderGRU } \end{aligned}$ <br> <br>
> Với $x_0^{\rightarrow}$ = \<sos>, $x_1^{\rightarrow}$ = gutten và $x_0^{\rightarrow}$ = \<eos>, $x_1^{\rightarrow}$ = morgen.

> Trước đây, chúng ta chỉ truyền 1 đầu vào (`embedded`) vào RNN, và thông báo với Pytorch rằng hãy khởi tạo forward và backward initial hidden states ($h_0^{\rightarrow}$ và $h_0^{\leftarrow}$) bằng tensor 0. Sau quá trình xử lý của RNN, ta thu được 2 context vectors, 1 là từ quá trình forward, $z^{\rightarrow}$ = $h_T^{\rightarrow}$ và 1 từ quá trình backward, $z^{\leftarrow}$ = $h_T^{\leftarrow}$.

> RNN trả về `outputs` và `hidden`.

> `outputs` có kích thước **[src len, batch size, hid dim * num directions]**. Phần tử `hid_dim` đầu tiên ở trục thứ 3 là hidden states từ top layer forward RNN, và phần tử `hid_dim` cuối cùng là từ top layer RNN backward. Ta có thể hiểu trục thứ 3 gồm forward và backward hidden states được ghép nối với nhau: $h_1$ = [$h_1^{\rightarrow}$;$h_T^{\leftarrow}$], $h_2$ = [$h_2^{\rightarrow}$;$h_{T-1}^{\leftarrow}$], ... và, ta ký hiệu tất cả các encoder hidden states (gồm hidden states từ forward và backward được ghép nối với nhau) là H = {$h_1, h_2, h_3, ..., h_T$}

> `hidden` có kích thước **[n layers * num directions, batch size, hid dim]**. Sử dụng **[-2,:,:]** cho ta hidden state của top layer forward RNN sau bước cuối cùng và **[-1,:,:]** cho ta hidden state của top layer backward RNN sau bước cuối cùng.

> Decoder là undirectional, nó chỉ cần một context vector z đóng vai trò initial hidden state $s_0$. Mà hiện tại ta đang có 2 context vector ($z^{\rightarrow}$ = $h_T^{\rightarrow}$ và $z^{\leftarrow}$ = $h_T^{\leftarrow}$). Đơn giản, ta chỉ cần ghép nối 2 vector này lại và cho đi qua một lớp fully connected để giảm về chiều của một vector (ở đây, activation function ta chọn là hàm tanh): <br> 
>> z = tanh(g($h_T^{\rightarrow}$, $h_T^{\leftarrow}$)) = tanh(g($z^{\rightarrow}$, $z^{\leftarrow}$)) = $s_0$.

> **NOTE**: Trong paper, người ta chỉ lấy hidden states từ backward RNN và đưa vào linear layer để thu context vector và decoder initial hidden state.

> Do model chúng ta cần quan sát toàn bộ source sentence nên chúng ta sẽ return `outputs`, stacked forward và backward hidden states của mỗi token trong source sentence. Ngoài ra, ta cần return `hidden` để làm initial hidden state ở decoder.

In [ ]:
class Encoder(nn.Module):
    def __init__(self,
                input_dim,
                emb_dim,
                hid_dim,
                n_layers,
                dropout):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, emb_dim)

        if n_layers > 1:
            self.rnn = nn.GRU(emb_dim, hid_dim, n_layers, dropout = dropout, bidirectional = True)
        else:
            self.rnn = nn.GRU(emb_dim, hid_dim, n_layers, bidirectional = True)

        self.fc = nn.Linear(hid_dim * 2, hid_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        # src = [src len, batch size]
        embedded = self.dropout(self.embedding(src))
        # embedded = [src len, batch size, emb dim]
        outputs, hidden = self.rnn(embedded)
        # outputs = [src len, batch size, hid dim * n directions]
        # hidden = [n layers * n directions, batch size, hid dim]

        # hidden is stacked [forward_1, backward_1, forward_2, backward_2, ...]
        # outputs are always from the last layer

        # hidden [-2, :, : ] is the last of the forwards RNN
        # hidden [-1, :, : ] is the last of the backwards RNN
        # initial decoder hidden is final hidden state of the forwards and backwards
        #  encoder RNNs fed through a linear layer
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))
        # outputs = [src len, batch size, hid dim * n directions]
        # hidden = [batch size, hid dim]
        return outputs, hidden

##### Attention

> Tiếp theo là attention layer. Layer này sẽ nhận đầu vào là previous hidden state của decoder $s_{t-1}$ và tất cả cách stacked foward và backward hidden states từ encoder, **H**. Layer sẽ trả về một attention vector $a_t$ có kích thước là chiều dài của source sentence, mỗi phần tử của vector nằm trong khoảng (0,1) và tổng các phần từ bằng 1.

> Bản chất là layer này sẽ lấy đối tượng đã được decode tính tới thời điểm hiện tại $s_{t-1}$ và tất cả những gì chúng ta đã encode, **H** để sinh ra một attention vector $a_t$, vector này biểu diễn việc model nên chú ý tới từ nào trong source sentence để đưa ra dự đoán chính xác cho từ tiếp theo $\hat{y}_{y+1}$.

> Đầu tiên, ta tính toán *energy* giữa previous decoder hidden state và encoder hidden states. Do hidden states của encoder là chuỗi gồm T tensors, và previous decoder hidden state là một tensor, điều đầu tiên ta cần thực hiện là lặp trong previous decoder hidden state **T** lần. Tiếp theo, chúng ta tính energy $E_t$ giữa chúng bằng cách ghép nối chúng lại với nhau và đưa vào linear layer (`attn`), đồng thời áp dụng tanh activation function. <br>
>> $E_t$ = $tanh(attn(s_{t-1}, H))$

> Điều này tương tự với việc ta thực hiện tính mức độ khớp giữa encoder hidden state và previous decoder hidden state.

> Hiện tại ta có **[dec hid dim, src len]** tensor cho mỗi example trong batch. Tuy nhiên, ta muốn tensor có kích thước **[src len]** cho mỗi example trong batch do attention phải có chiều dài bằng với chiều dài của source sentence. Để đạt được việc này, ta nhân `energy` với tensor v, **[1, dec hid dim]**.
>> $\hat{a}_t$ = v.$E_t$

> Ta có thể hiểu v là trọng số của tổng trọng số của energy đối với từng enoder hidden states. Trọng số này thông báo mức độ chứ ý đối với mỗi token trong source sequence. Các tham số trong v được khởi tạo ngẫu nhiên, và được học bởi model bằng backpropagation. Cần chú ý là v không phụ thuộc vào thời điểm và v được sử dụng cho nhiều thời điểm khác nhau. Ta implement v là một linear layer không có bias. <br>
> (Phần này hơi khó hiểu)

> Cuối cùng, để đảm bảo các phần tử trong attention vector nằm trong khoảng (0,1) và tổng các phần tử bằng 1, chúng ta truyền nó vào sofmax layer. <br>
>> $a_t$ = softmax($\hat{a}_t$)

> Biểu thức trên cho ta attention của toàn bộ source sentence!

> Dưới đây là hình biểu diễn việc tính toán attention khi $s_{t-1}$ = $s_0$ = z. Các khối màu green biểu diễn hidden states từ cả forward và backward RNNs, việc tính toán attention được khối màu pink xử lý.

![figure](./images/3.attention.png)

In [ ]:
class Attention(nn.Module):
    def __init__(self,
                enc_hid_dim,
                dec_hid_dim):
        super().__init__()
        self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
        self.v = nn.Linear(dec_hid_dim, 1, bias = False)

    def forward(self, hidden, encoder_outputs):
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]
        
        #repeat decoder hidden state src_len times
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #hidden = [batch size, src len, dec hid dim]
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim = 2))) 
        
        #energy = [batch size, src len, dec hid dim]

        attention = self.v(energy).squeeze(2)
        
        #attention= [batch size, src len]
        
        return F.softmax(attention, dim=1)

##### Decoder

> Tiếp theo, ta xây dựng decoder.

> Decoder chứa attention layer, `attention` với input là previous hidden state $s_{t-1}$, tất cả các encoder hidden states H và trả về attention vector $a_t$.

> Sau đó, ta sử dụng attention vector để tạo ra vector trọng số $w_t$, ký hiệu bởi `weighted`, là tổng trọng số của encoder hidden states H với $a_t$ là trọng số: <br> <br>
>> $w_t$ = $a_t$.H

> Embedded input word d($y_t$), source vector có trọng số $w_t$ và previous decoder hidden state $s_{t-1}$ đều được truyền vào decoder RNN, với d($y_t$) và $w_t$ được ghép nối với nhau. <Br> <br>
>> $s_t$ = DecoderGRU(d($y_t$), $w_t$, $s_{t-1}$)

> Cuối cùng ta truyền d($y_t$), $w_t$ và $s_t$ vào linear layer f để dự đoán từ tiếp theo trong target sentence, $\hat{y}_{t+1}$. <br> <br>
>> $\hat{y}_{t+1}$ = f(d($y_t$), $w_t$, $s_t$)

> Dưới đây là hình minh họa quá trình decoding từ đầu tiên trong quá trình dịch máy.

![figure](./images/3.first_example.png)

> Khối màu green là forward/backward encoder RNNs sinh ra H, khối màu red biểu diễn context vector z = $h_T$ = tanh(g($h_T^{\rightarrow}$, $h_T^{\leftarrow}$)) = tanh(g($z^{\rightarrow}$, $z^{\leftarrow}$)) = $s_0$. Khối màu blue là decoder RNN, sinh ra $s_t$, còn khối màu purple biểu diễn linear layer f, sinh ra $\hat{y}_{t+1}$ và khối màu orange biểu diễn việc tính toán có trọng số H từ $a_t$ và sinh ra $w_t$.

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()

        self.output_dim = output_dim
        self.attention = attention
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)
        
        self.fc_out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, encoder_outputs):
             
        #input = [batch size]
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
        
        a = self.attention(hidden, encoder_outputs)
                
        #a = [batch size, src len]
        
        a = a.unsqueeze(1)
        
        #a = [batch size, 1, src len]
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        weighted = torch.bmm(a, encoder_outputs)
        
        #weighted = [batch size, 1, enc hid dim * 2]
        
        weighted = weighted.permute(1, 0, 2)
        
        #weighted = [1, batch size, enc hid dim * 2]
        
        rnn_input = torch.cat((embedded, weighted), dim = 2)
        
        #rnn_input = [1, batch size, (enc hid dim * 2) + emb dim]
            
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        
        #output = [seq len, batch size, dec hid dim * n directions]
        #hidden = [n layers * n directions, batch size, dec hid dim]
        
        #seq len, n layers and n directions will always be 1 in this decoder, therefore:
        #output = [1, batch size, dec hid dim]
        #hidden = [1, batch size, dec hid dim]
        #this also means that output == hidden
        assert (output == hidden).all()
        
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        
        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim = 1))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden.squeeze(0)

##### Seq2Seq

> Đây là model đầu tiên ta có mà encdoer RNN và decoder RNN có hidden dimensions khác nhau. Tuy nhiên encoder là bidirectional. Yêu cầu này có thể gỡ bỏ bằng việc thay đổi điều kiện: `enc_dim * 2` thành `enc_dim * 2 if encoder_is_bidirectional else enc_dim`.

> Cơ bản, model seq2seq này khá giống với 2 model trước. Điểm khác biệt duy nhất là `encoder` trả về final hidden state từ cả forward và backward encoder RNN được truyền qua linear layer, được lấy làm initial hiddne state cho encoder, cũng như là ở mỗi hidden state. Ta cũng cần lưu ý là `hidden` và `encoder_outputs` được truyền vào decoder.

> Tóm tắt các bước của seq2seq model: <br>
>> * `outputs` tensor được khởi tạo để lưu trữ tất cả giá trị dự đoán, $\hat{Y}$.
>> * Source sequence **X** được truyền vào encoder để thu về z và H.
>> * Initial decoder hidden state được gán bằng context vector, $s_0$ = z = $h_T$.
>> * Ta sử dụng batch of \<sos> tokens là first input, $y_1$.
>> * Sau đó, ta thực hiện các công việc sau trong vòng lặp: <br>
>>> * Đưa input token $y_t$, previous hidden state $s_{t-1}$ và tất cả đầu ra của encoder H vào decoder.
>>> * Nhận về giá trị dự đoán $\hat{y}_{t+1}$ và new hidden state $s_t$.
>>> * Sau đó ta sẽ quyết định việc sử dụng teacher force hoặc không, thiết lập input tiếp theo với giá trị phù hợp.

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time
        
        batch_size = src.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #encoder_outputs is all hidden states of the input sequence, back and forwards
        #hidden is the final forward and backward hidden states, passed through a linear layer
        encoder_outputs, hidden = self.encoder(src)
                
        #first input to the decoder is the  tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            #insert input token embedding, previous hidden state and all encoder hidden states
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, encoder_outputs)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

### Training the Seq2Seq Model

> Khởi tạo tham số, encoder, decoder và seq2seq model.

In [ ]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, device).to(device)

> Khởi tạo các tham số bias bằng zeros tensors, các weights theo phân phối chuẩn N(0, 0.01).

In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
            
model.apply(init_weights)

> Tính toán số lượng tham số. Ta có thể thấy số lượng tham số tăng thêm khoảng 50% so với model trước.

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

In [ ]:
optimizer = optim.Adam(model.parameters())

In [ ]:
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

> Tạo train loop.

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

> Và evaluation loop, lưu ý là phải đưa model về chế độ eval để tắt dropout và teacher forcing, `mdoel.eval()`.

In [ ]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

> Timming function.

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

> Train and save model.

In [ ]:
# check folder exists, if not create it
import os
if not os.path.exists('models'):
    os.makedirs('models')

In [ ]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), './models/tut3-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

> Load and evaluate model.

In [ ]:
model.load_state_dict(torch.load('./models/tut3-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

> Trong notebook này, chúng ta đã cải thiện được model, tuy nhiên tăng gấp đôi thời gian tính toán.

> Trong notebook tiếp theo, chúng ta sẽ sử dụng kiến trúc model tương tự và áp dụng một vài thủ thuật cho các cấu trúc RNN - sử dụng packed padded sequences và masking. Chúng ta cũng sẽ implement code cho phép ta quan sát các input của RNN đang chú ý tới trong quá trình decoding the output.

## END